In [1]:
import xarray
import numpy as np
import pandas as pd
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
# ### States
path_LUH2 = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/states.nc'
luh2_states = xarray.open_dataset(path_LUH2, engine="netcdf4", decode_times=False)
luh2_states_worldwide = luh2_states.isel(time=slice(1000, 1173))
luh2_states_worldwide['time'] = pd.date_range(start="1850-01-01", end="2022-01-01", freq='YS')

c:\Users\HEFRY ANESTI\AppData\Local\Programs\Python\Python39\lib\site-packages\xarray\coding\times.py:167: SerializationWarning: Ambiguous reference date string: 850-01-01 0:0:0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-01-01 0:0:0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [3]:
luh2_added_states_worldwide = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-IMAGE-ssp119-2-1-f_gn_2015-2100.nc",\
                                        engine="netcdf4", decode_times=False)
luh2_added_trans = luh2_added_states_worldwide.drop_vars(['lat_bounds', 'lon_bounds', 'time_bnds'])
luh2_added_trans.coords['time'] = pd.date_range(start='2015-01-01', end='2100-01-01', freq='YS')

# ### Static
static = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/staticData_quarterdeg.nc'
luh2_static = xarray.open_dataset(static, engine="netcdf4")

country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code.xlsx", engine="openpyxl")
ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['name'])

ccode_convert = np.zeros((720, 1440), dtype="<U64")

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]

ccode_worldwide_int = luh2_static['ccode'].to_numpy().astype('int64')

for i in range(720):
    for j in range(1440):
        if ccode_worldwide_int[i][j] in ccode_dict.keys():
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "Unknown"

In [4]:
crop_class = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/Crop Classification_latest.xlsx", engine="openpyxl", skiprows=1)
crop_class = crop_class.drop('Unnamed: 0', axis=1)
crop_class.rename(columns={'FAO Crops': 'Item'}, inplace=True
                  )
fao_stat = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/fao_all_data.xlsx", engine="openpyxl")
fao_stat.dropna(axis=0, inplace=True)

fao_stat_area_harv = fao_stat[fao_stat['Element']=="Area harvested"]
fao_stat_area_harv.insert(9, 'True Value', fao_stat_area_harv['Value']/100, True)
faostat_area_harvested = fao_stat_area_harv.merge(crop_class, on='Item')

In [90]:
gfrac = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/GFRAC_GRAPC_Interpolation/GFRAC-RESAMPLE-NEW-1970-2100.nc", engine='netcdf4')
gfrac_ngfbfc = gfrac.coords['NGFBFC'].data.tolist()

gfrac_ngfbfc_ir_rf = []
for gfrac_cls in gfrac_ngfbfc:
    if  gfrac_cls[0:2] == 'RF' or gfrac_cls[0:2]== 'IR':
        gfrac_ngfbfc_ir_rf.append(gfrac_cls)

In [6]:
# fungsi interpolasi
def grid_level(kolom_sebelumnya, percent):
    return np.divide(np.multiply(kolom_sebelumnya, percent), 100)

def percent_change(gfrac, kolom_sekarang, kolom_sebelumnya):
    if kolom_sebelumnya != 0:
        result = np.multiply(np.nan_to_num(np.divide(kolom_sekarang, kolom_sebelumnya)), 100)
    else:
        result = np.nan
    return grid_level(gfrac, result)

In [7]:
def gfrac_interp(gfrac_class, lat_idx, country):

    arr = np.zeros((56, 1, 1, 1440), dtype="float64")

    arr[0] = np.array(([np.where(country in ccode_convert, gfrac['GFRAC_res'].isel(time=0).sel(NGFBFC=gfrac_class).to_numpy(), 0)[lat_idx]])).sum()
    arr[5] = np.array(([np.where(country in ccode_convert, gfrac['GFRAC_res'].isel(time=1).sel(NGFBFC=gfrac_class).to_numpy(), 0)[lat_idx]])).sum()
    arr[10] = np.array(([np.where(country in ccode_convert, gfrac['GFRAC_res'].isel(time=2).sel(NGFBFC=gfrac_class).to_numpy(), 0)[lat_idx]])).sum()
    arr[15] = np.array(([np.where(country in ccode_convert, gfrac['GFRAC_res'].isel(time=3).sel(NGFBFC=gfrac_class).to_numpy(), 0)[lat_idx]])).sum()
    arr[20] = np.array(([np.where(country in ccode_convert, gfrac['GFRAC_res'].isel(time=4).sel(NGFBFC=gfrac_class).to_numpy(), 0)[lat_idx]])).sum()
    arr[25] = np.array(([np.where(country in ccode_convert, gfrac['GFRAC_res'].isel(time=5).sel(NGFBFC=gfrac_class).to_numpy(), 0)[lat_idx]])).sum()
    arr[30] = np.array(([np.where(country in ccode_convert, gfrac['GFRAC_res'].isel(time=6).sel(NGFBFC=gfrac_class).to_numpy(), 0)[lat_idx]])).sum()
    arr[35] = np.array(([np.where(country in ccode_convert, gfrac['GFRAC_res'].isel(time=7).sel(NGFBFC=gfrac_class).to_numpy(), 0)[lat_idx]])).sum()
    arr[40] = np.array(([np.where(country in ccode_convert, gfrac['GFRAC_res'].isel(time=8).sel(NGFBFC=gfrac_class).to_numpy(), 0)[lat_idx]])).sum()
    arr[45] = np.array(([np.where(country in ccode_convert, gfrac['GFRAC_res'].isel(time=9).sel(NGFBFC=gfrac_class).to_numpy(), 0)[lat_idx]])).sum()
    arr[50] = np.array(([np.where(country in ccode_convert, gfrac['GFRAC_res'].isel(time=10).sel(NGFBFC=gfrac_class).to_numpy(), 0)[lat_idx]])).sum()
    arr[55] = np.array(([np.where(country in ccode_convert, gfrac['GFRAC_res'].isel(time=11).sel(NGFBFC=gfrac_class).to_numpy(), 0)[lat_idx]])).sum()
    
    years = list(np.linspace(1970, 2025, 56, dtype="int64"))
    gfrac_any_df = pd.DataFrame()

    columns_to_add = [
    pd.DataFrame({
        f'{year}': arr[i][0][0]
        }) for i, year in enumerate(years) 
    ]
    gfrac_any_df = pd.concat([gfrac_any_df] + columns_to_add, axis=1)

    gfrac_any_df_copy = gfrac_any_df.copy()
    columns_any_df = gfrac_any_df_copy.columns.astype(int)

    for column_idx, column in enumerate(columns_any_df[1:]):
        if gfrac_class[0:2] == 'RF':
            current_year_fao = (faostat_area_harvested[(faostat_area_harvested['Year'] == column)&(faostat_area_harvested['Area'] == country)&
                                    (faostat_area_harvested['IMAGE Classification']==gfrac_class.replace('RF ',''))]['True Value'].sum())
            
            previous_year_fao = (faostat_area_harvested[(faostat_area_harvested['Year'] == column - 1)&(faostat_area_harvested['Area'] == country)&
                                            (faostat_area_harvested['IMAGE Classification'] == gfrac_class.replace('RF ',''))]['True Value'].sum())

            current_year_gfrac = (gfrac_any_df_copy[f"{column}"])
            previous_year_gfrac = (gfrac_any_df_copy[f"{column - 1}"])

            if current_year_gfrac.any() == 0.0 or np.isnan(current_year_gfrac.any()): 
                gfrac_any_df_copy[f"{column}"] = percent_change(previous_year_gfrac, current_year_fao, previous_year_fao)
                arr[column_idx+1]  = percent_change(previous_year_gfrac, current_year_fao, previous_year_fao)
        
        elif gfrac_class[0:2] == 'IR':
            current_year_fao = (faostat_area_harvested[(faostat_area_harvested['Year'] == column)&(faostat_area_harvested['Area'] == country)&
                                    (faostat_area_harvested['IMAGE Classification']==gfrac_class.replace('IR ',''))]['True Value'].sum())
            
            previous_year_fao = (faostat_area_harvested[(faostat_area_harvested['Year'] == column - 1)&(faostat_area_harvested['Area'] == country)&
                                            (faostat_area_harvested['IMAGE Classification'] == gfrac_class.replace('IR ',''))]['True Value'].sum())
            
            current_year_gfrac = (gfrac_any_df_copy[f"{column}"])
            previous_year_gfrac = (gfrac_any_df_copy[f"{column - 1}"])
            
            if current_year_gfrac.any() == 0.0 or np.isnan(current_year_gfrac.any()): 
                gfrac_any_df_copy[f"{column}"] = percent_change(previous_year_gfrac, current_year_fao, previous_year_fao)
                arr[column_idx+1]  = percent_change(previous_year_gfrac, current_year_fao, previous_year_fao)

    gfrac_any_df_1970_2022 = gfrac_any_df_copy[gfrac_any_df_copy.columns[:-3]]

    arr_interp_carstock_any = []
    
    years_1970_2022 = list(np.linspace(1970, 2022, 53, dtype="int64"))
    
    for year in years_1970_2022:
        arr_interp_carstock_any.append([[gfrac_any_df_1970_2022[f'{year}']]])

    return np.array(arr_interp_carstock_any)


In [8]:
def convert_interp2arr(gfrac_class, country):
    
    interp_any = [] 
    for lat in range(720):
        interp_any.append(gfrac_interp(gfrac_class=f'{gfrac_class}', lat_idx=lat, country=country))
        
    arr_interp_any = np.array(interp_any)
    gfrac_interp_any_year = [] 
    
    for year in range(53):
        gfrac_interp_any_year.append([])
        for i in range(len(arr_interp_any)):
            gfrac_interp_any_year[year].append(arr_interp_any[i][year])

    gfrac_interp_any_year = np.array(gfrac_interp_any_year).reshape(53, 720, 1440)

    return gfrac_interp_any_year


In [ ]:
worldwide_gfrac_interp = []
for cty_idx, country in enumerate(list(np.unique(ccode_convert))):
    worldwide_gfrac_interp.append([])
    for gfrac_class in gfrac_ngfbfc_ir_rf:
        crops_interp_all_expand = np.expand_dims(convert_interp2arr(gfrac_class, country), axis=-1)
        worldwide_gfrac_interp[cty_idx].append(crops_interp_all_expand)

worldwide_gfrac_interp_arr = np.concatenate(worldwide_gfrac_interp, axis=-1)


In [ ]:
gfrac_interp_netcdf = xarray.Dataset({
        "gfrac_interp":(["country", "time", "lat", "lon", "ngfbfc"], worldwide_gfrac_interp_arr)
    },
    coords={
        "NGFBFC": np.array(gfrac_ngfbfc_ir_rf, dtype='<U35'),
        "country": list(np.unique(ccode_convert)),
        "time":pd.date_range("1970-01-01", "2022-01-01", freq='YS'),
        "lon":luh2_states_worldwide.coords["lon"].to_numpy(),
        "lat":luh2_states_worldwide.coords["lat"].to_numpy()
    })

gfrac_interp_netcdf.to_netcdf("c:/Users/HEFRY ANESTI/Downloads/GFRAC-INTERPOLATION-WORLDWIDE-1970_2022.nc", mode='w', format="NETCDF4")